# Imports

In [1]:
import csv
import os
from utils import get_id_list, get_data, write_file

from dotenv import load_dotenv
load_dotenv()

True

# Grabbing All Movie IDs

In [2]:
TMBD_API_KEY = os.getenv('TMBD_API_KEY')
YEAR = 2023

# Grab list of ids for all films made in {YEAR}
movie_list = get_id_list(TMBD_API_KEY, YEAR)
print(movie_list)

['792307', '1072790', '940551', '984249', '787699', '799710', '609681', '1211483', '949429', '1029575', '1022796', '1079394', '980137', '572802', '872585', '753342', '346698', '385687', '1078012', '880100', '667538', '1183905', '502356', '508883', '1010581', '299054', '1189798', '695721', '1042279', '466420', '940721', '603692', '1101582', '897087', '676727', '982940', '976573', '1076011', '569094', '848326', '901362', '575264', '1115377', '838240', '467244', '298618', '507089', '1075794', '615656', '893723', '967585', '944401', '926393', '670292', '1105832', '906126', '915935', '447365', '1046090', '678512', '979097', '899445', '1058634', '891699', '988078', '1058694', '980489', '927107', '335977', '964592', '565770', '762430', '760774', '496450', '447277', '666277', '1001500', '1071215', '840430', '1053592', '520758', '994108', '842675', '640146', '459003', '968051', '999087', '493529', '976893', '614930', '951491', '1040148', '594767', '820525', '955659', '1168733', '1008042', '1025

# Writing all IDs to CSV file

In [3]:
FILE_NAME = '2023_movie_collection_data.csv'
list_of_attributes = [
    'Movie_Name', 
    'Runtime',
    'Original_Language', 
    'Overview',
    'Genre',
    'Keywords'
    ]

# Creating file
csvFile = open(FILE_NAME, 'w')
csvwriter = csv.writer(csvFile)
csvwriter.writerow(list_of_attributes)
csvFile.close()

# Iterate through list of ids to get data
for id in movie_list:
    text = get_data(TMBD_API_KEY, id)
    write_file(FILE_NAME, text)

# Random Testing

In [13]:
import requests
import json 

API_key = TMBD_API_KEY
# Movie_ID = '807172' # Exorcist
Movie_ID =  '1072790' # Anyone but you

url = 'https://api.themoviedb.org/3/movie/' + \
    Movie_ID+'?api_key='+API_key+'&language=en-US&append_to_response=keywords'

response = requests.get(url)
dict = response.json()
pretty_json = json.dumps(dict, indent=4)

# print(pretty_json)
all_genres = dict['genres']
genre_str = ""

for genre in all_genres:
    genre_str += genre['name'] + ", "

genre_str = genre_str[:-2]

all_keywords = dict['keywords']['keywords']
keyword_str = ""
for keyword in all_keywords:
    keyword_str += keyword['name'] + ", "

keyword_str = keyword_str[:-2]

print(keyword_str)

australia, hiking, college, romantic comedy, based on play or musical, shakespeare in modern dress, aftercreditsstinger, duringcreditsstinger, fake dating, sister's wedding, much ado about nothing
